# Taylor Polynomial Approximations

Adam Rumpf

Created 4/17/21

Based on a <a href="https://github.com/adam-rumpf/mathematica-class-demonstrations#taylor-and-fourier-series-approximations" target="_blank">Mathematica class demonstration</a>.

This is a standalone widget for playing around with Taylor polynomial approximations of various functions. See the full notebook [here](./taylor-series.ipynb).

[Main Project Page](.././index.ipynb)

In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

# Define parameters
INF = 100 # out-of-bounds value
LIM = 10 # axis bounds

# Define functions and derivatives
def d_sin(x, n=0):
    """n-th derivative of sin(x)."""
    
    r = n % 4
    if r == 0:
        return np.sin(x)
    elif r == 1:
        return np.cos(x)
    elif r == 2:
        return -np.sin(x)
    elif r == 3:
        return -np.cos(x)

def d_cos(x, n=0):
    """n-th derivative of cos(x)."""
    
    return d_sin(x, n=n+1)

def d_exp(x, n=0):
    """n-th derivative of exp(x)."""
    
    return np.exp(x)

def d_log(x, n=0):
    """n-th derivative of log(x+1)."""
    
    if n == 0:
        return np.log(x+1)
    else:
        return (np.math.factorial(n-1) * (-1)**(n+1)) / (x+1)**n

def d_bell(x, n=0):
    """n-th derivative of a bell curve."""
    
    out = np.exp(-(x**2)/5) * 5**(-(n-1))
    if n == 0:
        return out
    elif n == 1 or n == 2:
        out *= -2
        if n == 1:
            out *= x
        elif n == 2:
            out *= 5 - 2*(x**2)
    elif n == 3 or n == 4:
        out *= 4
        if n == 3:
            out *= (15 - 2*(x**2))*x
        elif n == 4:
            out *= 75 - 60*(x**2) + 4*(x**4)
    elif n == 5 or n == 6:
        out *= -8
        if n == 5:
            out *= (375 - 100*(x**2) + 4*(x**4))*x
        elif n == 6:
            out *= 1875 - 2250*(x**2) + 300*(x**4) - 8*(x**6)
    elif n == 7 or n == 8:
        out *= 16
        if n == 7:
            out *= (13125 - 5250*(x**2) + 420*(x**4) - 8*(x**6))*x
        elif n == 8:
            out *= 65625 - 105000*(x**2) + 21000*(x**4) - 1120*(x**6) + 16*(x**8)
    elif n == 9 or n == 10:
        out *= -32
        if n == 9:
            out *= (590625 - 315000*(x**2) + 37800*(x**4) + 1440*(x**6) + 16*(x**8))*x
        elif n == 10:
            out *= 2953125 - 5906250*(x**2) + 1575000*(x**4) - 126000*(x**6) + 3600*(x**8) - 32*(x**10)
    return out

def d_poly(x, n=0):
    """n-th derivative of a polynomial function."""
    
    if n == 0:
        return 0.35 + x*(0.16 + x*(-0.1875 + x*(0.005 + x*0.0025)))
    elif n == 1:
        return 0.16 + x*(-0.375 + x*(0.015 + x*0.01))
    elif n == 2:
        return -0.375 + x*(0.03 + x*0.03)
    elif n == 3:
        return 0.03 + x*0.06
    elif n == 4:
        return 0.06 + x*0.0
    else:
        return x*0.0

def d_ratio(x, n=0):
    """n-th derivative of 1/(x+1)."""
    
    return ((-1)**n)/((x+1)**(n+1))

# Define a dictionary of function definitions
func = {}
func["sine"] = d_sin
func["cosine"] = d_cos
func["exponential"] = d_exp
func["logarithm"] = d_log
func["bell curve"] = d_bell
func["polynomial"] = d_poly
func["rational"] = d_ratio

# Define a dictionary of function name strings
func_name = {}
func_name["sine"] = "$\sin x$"
func_name["cosine"] = "$\cos x$"
func_name["exponential"] = "$e^x$"
func_name["logarithm"] = "$\log(x+1)$"
func_name["bell curve"] = "$5e^{-x^2/5}$"
func_name["polynomial"] = "$0.35 + 0.16x - 0.1875x^2 + 0.005x^3 + 0.0025x^4$"
func_name["rational"] = "$1/(x+1)$"

# Define Taylor polynomial
def taylor(x, fname, a, n):
    """Taylor polynomial for a given function.
    
    Positional arguments:
    x - input value
    fname - key from 'func' dictionary
    a - center
    n - polynomial degree
    """
    
    out = 0.0 # output value
    
    # Add terms of Taylor polynomial
    for i in range(n+1):
        out += (func[fname](a, n=i) / np.math.factorial(i)) * (x-a)**i
    
    return out

# Set up plot
fig, ax = plt.subplots()
xbase = np.linspace(-LIM, LIM, 101) # base x-values

# Draw plot lines
@widgets.interact(fname=func.keys(), a=(-LIM, LIM, 0.05), n=(0, 10, 1))
def update1(fname="sine", a=0.0, n=1):
    
    global ax
    a0 = a
    
    # Generate function values
    if fname == "logarithm":
        x = np.linspace(-0.99, LIM, 101)
        a0 = max(a0, -0.9)
    elif fname == "rational":
        x = np.linspace(-LIM, LIM, 100)
        if a0 == -1.0:
            a0 += 0.05
    else:
        x = np.linspace(-LIM, LIM, 101)
    y = np.zeros_like(x)
    
    # Redraw plot
    ax.clear()
    ax.set_xlim([-LIM, LIM])
    ax.set_ylim([-LIM, LIM])
    ax.set_aspect(1)
    plt.title(func_name[fname])
    ax.grid(False)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.plot(x, func[fname](x), color="C0")
    y = taylor(xbase, fname, a0, n)
    ax.plot(xbase, y, color="C1")
    ax.plot(a0, func[fname](a0), color="C1", marker=".", markersize=10)
    if fname in {"logarithm", "rational"}:
        ax.plot([-1, -1], [-INF, INF], color="white")
        ax.plot([-1, -1], [-INF, INF], color="black", linestyle="dashed")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='fname', options=('sine', 'cosine', 'exponential', 'logarithm', 'be…